[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/latent-consistency-model-colab/blob/main/wip/lcm_sdxl_controlnet_openpose_draw_colab.ipynb)

In [ ]:
!pip install git+https://github.com/huggingface/diffusers -U
!pip install -q controlnet-aux transformers accelerate peft gradio==3.50.2
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

In [ ]:
from diffusers import StableDiffusionXLControlNetPipeline, UNet2DConditionModel, ControlNetModel, LCMScheduler
import torch

controlnet = ControlNetModel.from_pretrained("thibaud/controlnet-openpose-sdxl-1.0", torch_dtype=torch.float16)
unet = UNet2DConditionModel.from_pretrained("latent-consistency/lcm-sdxl", torch_dtype=torch.float16, variant="fp16",)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", unet=unet, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None,).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

In [ ]:
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image, make_image_grid

openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
original_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/person.png")
openpose_image = openpose(original_image)

In [ ]:
image = pipe(f"cute girl", image=openpose_image, num_inference_steps=4).images[0]
make_image_grid([original_image, openpose_image, image], rows=1, cols=3)